In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-sentiment-analysis/testData.tsv
/kaggle/input/imdb-sentiment-analysis/labeledTrainData.tsv
/kaggle/input/imdb-sentiment-analysis/sampleSubmission.csv
/kaggle/input/imdb-sentiment-analysis/unlabeledTrainData.tsv


In [2]:
!pip install evaluate
import os
import sys
import logging
import datasets
import evaluate

import pandas as pd
import numpy as np

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudn

2025-05-14 03:55:31.413088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747194931.637162      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747194931.706124      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 预处理数据

In [3]:
trainDataPath = r"/kaggle/input/imdb-sentiment-analysis/labeledTrainData.tsv"
testDataPath = r"/kaggle/input/imdb-sentiment-analysis/testData.tsv"

train = pd.read_csv(trainDataPath, header=0, delimiter="\t", quoting=3)
test = pd.read_csv(testDataPath, header=0, delimiter="\t", quoting=3)

print(train.shape)

(25000, 3)


In [4]:
train, val = train_test_split(train, test_size=0.2)
print(train.shape, val.shape)

train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)
print(train_dataset)

(20000, 3) (5000, 3)
Dataset({
    features: ['label', 'text'],
    num_rows: 20000
})


In [5]:
checkpoint = "microsoft/deberta-v2-xxlarge"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
print(tokenized_train)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})


## 创建模型

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType
# 定义Config
prompt_tuning_init_text = "Classify if the movie review is positive or negative.\n"
peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=len(tokenizer(prompt_tuning_init_text)["input_ids"]),
    prompt_tuning_init_text=prompt_tuning_init_text,
    tokenizer_name_or_path=checkpoint,
)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xxlarge and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 23,042 || all params: 1,566,936,580 || trainable%: 0.0015


## 定义Trainer

In [8]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
training_args = TrainingArguments(
        run_name="deberta-prompt",
        output_dir='./deberta-v2-xxlarge-prompt',  
        learning_rate=1e-4,
        num_train_epochs=3,  
        per_device_train_batch_size=1,  
        per_device_eval_batch_size=1,  
        weight_decay=0.01,
        save_strategy="epoch",
        eval_strategy="epoch",
        report_to="none",
)

In [10]:
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.945800,0.739726,0.861200
2,0.639500,0.502881,0.910400
3,0.535400,0.488399,0.916800


TrainOutput(global_step=60000, training_loss=0.7909505503336589, metrics={'train_runtime': 30717.9268, 'train_samples_per_second': 1.953, 'train_steps_per_second': 1.953, 'total_flos': 1.3027468814669878e+17, 'train_loss': 0.7909505503336589, 'epoch': 3.0})

## 预测,保存输出

In [11]:
import os

# 确保result文件夹存在
# 自动创建且不报错
os.makedirs("/kaggle/working/result", exist_ok=True)  # 自动创建且不报错（

prediction_outputs = trainer.predict(tokenized_test)
test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()
print(test_pred)

result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/kaggle/working/result/deberta_prompt.csv", index=False, quoting=3)

[1 0 1 ... 0 1 1]
